In [1]:
import os
import psycopg2 as ps
import pandas as pd

In [2]:
conn = ps.connect(
    host='localhost',
    port='5432',
    dbname='chinook',
    user='chinook',
    password='password')

# Senior-most employee based on job title

In [5]:
df = pd.read_sql_query("""SELECT 
                            CONCAT(first_name, ' ', last_name) AS name,
                            title
                        FROM
                            employee
                        WHERE 
                            reports_to IS NULL""",
                        con=conn)
df

C:\Users\sethi\AppData\Local\Temp\ipykernel_192076\2139371420.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query("""SELECT


,name,title
0,Andrew Adams,General Manager


Andrew Adams holds the highest position as Senior General Manager 
<br><br>
Actionable Insights:
1. Involve Madan Mohan in high-level strategic decisions.
2. Develop a succession plan for this critical role.
3. Analyze the career path that led to this position to guide future leadership development.


# Employee Hierarchy

In [14]:
df = pd.read_sql_query("""SELECT
                            CONCAT(m.first_name, ' ', m.last_name) AS name,
                            m.title,
                            COUNT(e.employee_id) AS reporting_count,
                            ARRAY_AGG(CONCAT(e.first_name, ' ', e.last_name)) AS reporting
                        FROM
                            employee m
                        JOIN
                            employee e
                        ON
                            m.employee_id = e.reports_to
                        GROUP BY
                            name,
                            m.title
                        ORDER BY
                            name""",
                        con=conn)
df

C:\Users\sethi\AppData\Local\Temp\ipykernel_192076\1443834522.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query("""SELECT


,name,title,reporting_count,reporting
0,Andrew Adams,General Manager,2,"[Michael Mitchell, Nancy Edwards]"
1,Michael Mitchell,IT Manager,2,"[Robert King, Laura Callahan]"
2,Nancy Edwards,Sales Manager,3,"[Steve Johnson, Jane Peacock, Margaret Park]"


<b>Hierarchy Structure:</b>
<br>
The data illustrates a managerial hierarchy within the organization. 
<br>
At the top, you have Andrew Adams as the General Manager, who oversees two managers: Michael Mitchell and Nancy Edwards.
<br>
Michael Mitchell holds the title of IT Manager and has two direct reports: Robert King and Laura Callahan.
<br>
Nancy Edwards is the Sales Manager with three direct reports: Steve Johnson, Jane Peacock, and Margaret Park.

# Best performing sales rep based on customer count

In [15]:
df = pd.read_sql_query("""SELECT
                            CONCAT(e.first_name, ' ', e.last_name) AS name,
                            e.title,
                            COUNT(c.customer_id) AS customer_count
                        FROM
                            employee e
                        LEFT JOIN
                            customer c
                        ON
                            e.employee_id = c.support_rep_id
                        GROUP BY
                            name, e.title
                        HAVING
                            COUNT(c.customer_id) > 0
                        ORDER BY
                            customer_count DESC""",
                        con=conn)
df

C:\Users\sethi\AppData\Local\Temp\ipykernel_192076\3659069264.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query("""SELECT


,name,title,customer_count
0,Jane Peacock,Sales Support Agent,21
1,Margaret Park,Sales Support Agent,20
2,Steve Johnson,Sales Support Agent,18


# Customer who spend the most

In [11]:
df = pd.read_sql_query("""SELECT
                            CONCAT(c.first_name, ' ', c.last_name) AS customer_name,
                            SUM(total) AS spend
                        FROM
                            customer c
                        LEFT JOIN
                            invoice i
                        ON
                            c.customer_id = i.customer_id
                        GROUP BY
                            customer_name
                        ORDER BY
                            spend DESC
                        LIMIT 10""",
                        con=conn)
df

C:\Users\sethi\AppData\Local\Temp\ipykernel_192076\1187535012.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query("""SELECT


,customer_name,spend
0,Helena Holý,49.62
1,Richard Cunningham,47.62
2,Luis Rojas,46.62
3,Ladislav Kovács,45.62
4,Hugh O'Reilly,45.62
5,Frank Ralston,43.62
6,Fynn Zimmermann,43.62
7,Julia Barnett,43.62
8,Victor Stevens,42.62
9,Astrid Gruber,42.62


In [13]:
df = pd.read_sql_query("""SELECT
                            billing_city AS city,
                            SUM(total) AS revenue
                        FROM
                            invoice
                        GROUP BY
                            billing_city
                        ORDER BY
                            revenue DESC
                        LIMIT 10""",
                        con=conn)
df

C:\Users\sethi\AppData\Local\Temp\ipykernel_192076\923522913.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query("""SELECT


,city,revenue
0,Prague,90.24
1,Paris,77.24
2,Mountain View,77.24
3,São Paulo,75.24
4,Berlin,75.24
5,London,75.24
6,Fort Worth,47.62
7,Santiago,46.62
8,Dublin,45.62
9,Budapest,45.62
